In [91]:
# basic libraries
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
import matplotlib.pyplot as plt

import gensim
from nltk.tokenize.toktok import ToktokTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences


# n-gram
import math
from collections import Counter, defaultdict
from typing import List # somehow python 3.9 requires

import nltk
import numpy as np
import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer
# from sklearn.metrics import precision_recall_fscore_support
from tqdm import tqdm


from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_fscore_support


"""
Config
跑之前要改config = writing_patterns/Quantity/Both

"""

writing_patterns = ['title_special_chars', 'text_special_chars',
       'title_dets', 'text_dets', 'title_capitals', 'text_capitals',
       'title_short_sents', 'text_short_sents', 'title_long_sents',
       'text_long_sents']

Quantity = ['title_words', 'text_words',
       'title_sents', 'text_sents', 'title_words_per_sent',
       'text_words_per_sent', 'title_verbs', 'text_verbs', 'title_adjs',
       'text_adjs', 'title_advs', 'text_advs', 'title_rate_adjs_and_advs',
       'text_rate_adjs_and_advs','title_fog', 'text_fog', 'title_smog', 'text_smog',
       'title_ari', 'text_ari']

Both = ['title_words', 'text_words',
       'title_sents', 'text_sents', 'title_words_per_sent',
       'text_words_per_sent', 'title_verbs', 'text_verbs', 'title_adjs',
       'text_adjs', 'title_advs', 'text_advs', 'title_rate_adjs_and_advs',
       'text_rate_adjs_and_advs', 'title_special_chars', 'text_special_chars',
       'title_dets', 'text_dets', 'title_capitals', 'text_capitals',
       'title_short_sents', 'text_short_sents', 'title_long_sents',
       'text_long_sents', 'title_fog', 'text_fog', 'title_smog', 'text_smog',
       'title_ari', 'text_ari']


config = Both



In [92]:
train = pd.read_csv('/Users/kiyas/Downloads/dataset (with valid)/train.csv')
test = pd.read_csv('/Users/kiyas/Downloads/dataset (with valid)/test.csv')


## skip-gram

In [3]:

#%% train skip-gram model
tokenizer = ToktokTokenizer()
corpus = [tokenizer.tokenize(document) for document in train['title']]
model = gensim.models.Word2Vec(corpus, min_count = 5, window = 5, sg = 1) # sg -> skip gram = true
model.train(corpus, total_examples=len(corpus), epochs = 5)

vector = model.wv


In [4]:
words = model.wv.index_to_key
wvs = model.wv[words]
dic = {}
for i,f in enumerate(words):
    dic[f] = wvs[i]

---

In [59]:
x_train = []

for i,f in enumerate(train['text']):
    one = np.zeros((500, 1))
    corpus = [['[CLS]'] + tokenizer.tokenize(f)] + [['[CLS]'] + tokenizer.tokenize(train['title'][i])]
        
    if len(corpus[0]) < 500:
        for i in range(500-len(corpus[0])):
            corpus[0].append('0')

    one=[]
    for ii,ff in enumerate(corpus[0][0:500]):
        try:
            one.append(dic[ff].mean())
        except:
            one.append(0)
            
    # print(one)
    x_train.append(one)


In [57]:

x_test = []
for i,f in enumerate(test['text']):
    corpus = [['[CLS]'] + tokenizer.tokenize(f)] + [['[CLS]'] + tokenizer.tokenize(test['title'][i])]
        
    if len(corpus[0]) < 500:
        for i in range(500-len(corpus[0])):
            corpus[0].append('0')

    one=[]
    for ii,ff in enumerate(corpus[0][0:500]):
        try:
            one.append(dic[ff].mean())
        except:
            one.append(0)
            
    # print(one)
    x_test.append(one)


In [66]:
x_train = np.array(x_train)
x_test = np.array(x_test)

In [75]:
# 後面有feature的
x_train = np.concatenate([x_train, train[config]], axis=1)
x_test = np.concatenate([x_test, test[config]], axis=1)

In [62]:
y_train = train['label']
y_test = test['label']

## naive bayes

In [33]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef

In [78]:
nb_model = GaussianNB()
nb_model.fit(x_train, y_train)

F1 score: 0.5833688906891188, Precision: 0.6198457918963036, Recall: 0.6003355766401883
accuracy: 0.6003177685585666
mcc: 0.2193152674832797


In [ ]:
y_predicted = nb_model.predict(x_test)
precision, recall, f1, support = precision_recall_fscore_support(y_test, y_predicted, average='macro', zero_division=1)
print(f"F1 score: {f1}, Precision: {precision}, Recall: {recall}")

acc = accuracy_score(y_test, y_predicted)
print(f"accuracy: {acc}")

mcc = matthews_corrcoef(y_test, y_predicted)
print(f"mcc: {mcc}")